# 📊 Tam Model Karşılaştırması

Bu notebook tüm **ÜCRETSİZ** modelleri karşılaştırır.

---

In [ ]:
import sys, os, gc
import torch
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm

PROJECT_ROOT = os.path.dirname(os.getcwd())
sys.path.insert(0, PROJECT_ROOT)

from src.utils import PromptManager, print_model_table, FREE_MODELS, COLAB_MODELS
from src.evaluation.nlp_metrics import BLEUEvaluator, ROUGEEvaluator

print(f"CUDA: {torch.cuda.is_available()}")
print_model_table()

## 🆓 Sadece Ücretsiz Modeller

In [ ]:
print("\n✅ Colab T4 için test edilecek modeller:")
for m in COLAB_MODELS:
    print(f"  🆓 {m}")

In [ ]:
# Model yükleyiciler
def load_qwen2_vl():
    from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2-VL-2B-Instruct", torch_dtype=torch.float16, device_map="auto"
    )
    return model, processor

def run_qwen2_vl(model, processor, image, prompt):
    messages = [{"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": prompt}
    ]}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to("cuda")
    output_ids = model.generate(**inputs, max_new_tokens=512)
    return processor.batch_decode(output_ids, skip_special_tokens=True)[0]

def clear_gpu():
    gc.collect()
    torch.cuda.empty_cache()
    print(f"GPU: {torch.cuda.memory_allocated()/1e9:.1f} GB")

In [ ]:
# Test ayarları
test_image_path = os.path.join(PROJECT_ROOT, "test_xray.png")
image = Image.open(test_image_path).convert("RGB")

pm = PromptManager()
prompt = pm.get_prompt("rrg", "detailed").user_prompt

reference = "FINDINGS: Normal heart size. Clear lungs. IMPRESSION: Normal chest X-ray."

In [ ]:
# Karşılaştırma
MODELS = [
    ("Qwen2-VL-2B", load_qwen2_vl, run_qwen2_vl),
]

results = []

for name, load_fn, run_fn in MODELS:
    print(f"\n🔄 {name}...")
    try:
        model, processor = load_fn()
        output = run_fn(model, processor, image, prompt)
        results.append({"model": name, "output": output})
        print(f"✅ {len(output)} karakter")
        del model, processor
        clear_gpu()
    except Exception as e:
        print(f"❌ {e}")

In [ ]:
# Değerlendirme
bleu = BLEUEvaluator()
rouge = ROUGEEvaluator()

eval_results = []
for r in results:
    scores = bleu.compute([r["output"]], [reference])
    scores.update(rouge.compute([r["output"]], [reference]))
    eval_results.append({"model": r["model"], **scores})

pd.DataFrame(eval_results)

In [ ]:
# Sonuçları kaydet
import json
with open(os.path.join(PROJECT_ROOT, "results", "comparison_results.json"), "w") as f:
    json.dump({"results": results, "evaluation": eval_results}, f, indent=2)
print("✅ Kaydedildi!")